In [1]:
import copy
import numpy as np
import time
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torchvision

from timm.loss import LabelSmoothingCrossEntropy
from torch.optim import lr_scheduler
from torch.utils.data import random_split
from torchvision import datasets, models
from torchvision import transforms as T

/Users/zfac230/.venv/botanica/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [3]:
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt


trainset = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

testset = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=True)

In [4]:
classes = trainset.classes
num_class = len(classes)

In [5]:
class TinyModel(torch.nn.Module):

    def __init__(self):
        super(TinyModel, self).__init__()

        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [6]:
pso_pop = 7
fa_pop = 3
dim = 3
w = 0.5
alpha_0 = 0.9
alpha_f = 0.1
df = 0.5
states = np.array([[1.3,0.8],
                   [1.2,1.2],
                   [0.6,1.5]])
maxiter = 10
fa_options = {'bmin':1, 'gamma':0.8, 'alpha':0.2}
search_bounds = (np.array([0] * dim), np.array([1] * dim))

In [7]:
model1 = TinyModel()
model2 = TinyModel()
model3 = TinyModel()
models = [model1, model2, model3]
weights = [0.1, 0.5, 0.8]
threshold=0.5
epochs=3

In [8]:
from model_selection import _model_wrapper
from architecture import transformer_ensemble_weighted
_model_wrapper(weights, transformer_ensemble_weighted, models, threshold)

chosen models:  [1, 2]


transformer_ensemble_weighted(
  (sftmx): Softmax(dim=1)
)

In [9]:
from model_selection import fitness_wrapper 

In [10]:
fitness_wrapper(weights, transformer_ensemble_weighted, models, threshold, trainloader, trainset, testloader, testset, mps_device)

chosen models:  [1, 2]
TinyModel
TinyModel


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 938/938 [00:02<00:00, 347.57it/s]


<class 'torch.Tensor'>
Test Loss: 0.0012
top_k acc: 0.0942, 0.3172, 0.5000
train scores:  (tensor(0.0942, device='mps:0'), tensor(0.3172, device='mps:0'), tensor(0.5000, device='mps:0'), 0.0012364606221516928)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:00<00:00, 334.91it/s]

<class 'torch.Tensor'>
Test Loss: 0.0037
top_k acc: 0.0954, 0.3184, 0.5000
val scores:  (tensor(0.0954, device='mps:0'), tensor(0.3184, device='mps:0'), tensor(0.5000, device='mps:0'), 0.003676866912841797)


0.003676866912841797

In [11]:
def swarm_wrapper(position):
  #position to parameter conversion not necessary here
  # ub = [0.001, 0.95, 0.001] # Upperbounds [ub_lr, ub_b1, ub_w]
  # lb = [0.00001, 0.5, 0.0000001] # Lowerbounds [lb_lr, lb_b1, lb_w]
  # param_bounds = (lb,ub)
  # parameters = convert_pos_to_params(position, param_bounds)
  fitness = np.empty(len(position))

  for i in range(len(position)):
    print("postition :", position[i])
    fitness[i] = fitness_wrapper(position[i], transformer_ensemble_weighted, models, threshold, trainloader, trainset, testloader, testset, mps_device)
    print("fitness", fitness[i])

  print("fitness", fitness)
  return fitness

In [12]:
position = np.array([[0.1, 0.1, 0.1],
                     [0.5, 0.6, 0.7],
                     [0.1, 0.6, 0.4]
                    ])

In [13]:
swarm_wrapper(position)

postition : [0.1 0.1 0.1]
ERROR: no models selected
fitness inf
postition : [0.5 0.6 0.7]
chosen models:  [0, 1, 2]
TinyModel
TinyModel
TinyModel


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 938/938 [00:02<00:00, 351.30it/s]


<class 'torch.Tensor'>
Test Loss: 0.0012
top_k acc: 0.0942, 0.3172, 0.5000
train scores:  (tensor(0.0942, device='mps:0'), tensor(0.3172, device='mps:0'), tensor(0.5000, device='mps:0'), 0.0012237332661946614)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:00<00:00, 359.71it/s]


<class 'torch.Tensor'>
Test Loss: 0.0037
top_k acc: 0.0954, 0.3184, 0.5000
val scores:  (tensor(0.0954, device='mps:0'), tensor(0.3184, device='mps:0'), tensor(0.5000, device='mps:0'), 0.0036703155517578126)
fitness 0.0036703155517578126
postition : [0.1 0.6 0.4]
chosen models:  [1]
TinyModel


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 938/938 [00:02<00:00, 359.36it/s]


<class 'torch.Tensor'>
Test Loss: 0.0012
top_k acc: 0.1000, 0.3008, 0.5000
train scores:  (tensor(0.1000, device='mps:0'), tensor(0.3008, device='mps:0'), tensor(0.5000, device='mps:0'), 0.0012390932718912761)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:00<00:00, 358.83it/s]

<class 'torch.Tensor'>
Test Loss: 0.0037
top_k acc: 0.1000, 0.3021, 0.5000
val scores:  (tensor(0.1000, device='mps:0'), tensor(0.3021, device='mps:0'), tensor(0.5000, device='mps:0'), 0.0037070682525634766)
fitness 0.0037070682525634766
fitness [       inf 0.00367032 0.00370707]


array([       inf, 0.00367032, 0.00370707])